open the following link with a new tab

<a href="https://colab.research.google.com/github/nzhinusoftcm/review-on-collaborative-filtering/blob/master/5.Regularized_SVD(Matrix_Factorization).ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Regularized SVD (Matrix Factorization)

<b>User-based</b> and <b>Item-based</b> collaborative Filtering recommender systems suffer from <i>data sparsity</i> and <i>scalability</i> for online recommendations. <b>Matrix Factorization</b> helps to address these drawbacks of memory-based collaborative filtering by reducing the dimension of the rating matrix $R$.

The movielen lasted small dataset has 100k ratings of $m=610$ users on $n=9724$ items. The rating matrix in then a $m\times n$ matrix (i.e $R\in \mathbb{R}^{m\times n}$). The fact that users usually interact with less than $1\%$ of items leads the rating matrix $R$ to be highly sparse. For example, the degree of sparsity of the movielen lasted small dataset is 

\begin{equation}
sparsity = 100 - \frac{\text{total # ratings}}{m \times n} = 100 - \frac{100000}{610\times 9724} = 98,3\%
\end{equation}

This means that in this dataset, a user has interacted with less than $2\%$ of items. To reduce the dimension of the rating matrix $R$, Matrix Factorization (MF) mappes both users and items to a joint latent factor space of dimensionality $k$ such that user-item interactions are modeled as inner products in that space <a href='https://ieeexplore.ieee.org/document/5197422'>(Yehuda Koren et al., 2009)</a>. MF then decomposes $R$ in two matrices as follows :

\begin{equation}
R = Q^\top P
\end{equation}

Where $P \in \mathbb{R}^{m\times k}$ represents latent factors of users and $Q \in \mathbb{R}^{n\times k}$ is the latent factors of items. Each line of $P$, say $p_u \in \mathbb{R}^k$ denotes the taste of user $u$ and each $q_i \in \mathbb{R}^k$ the features of item $i$. The dot product between $p_u$ and $q_i$ will be the rating prediction of user $u$ on item $i$ :

\begin{equation}
\hat{r}_{u,i} = q_{i}^{\top} p_u.
\end{equation}

Figure 1 presents an example of decomposition of $R$ into two matrices $P$ and $Q$.

<img src="recsys/img/MF.png">
<br>
<center><b>Figure 1</b>: Decomposition of $R$ into $P$ and $Q$</center>


To learn the latent factors $p_u$ and $q_i$, the system minimizes the regularized squared error on the set of known ratings. The cost function $J$ is defined as follows : 

\begin{equation}
J = \frac{1}{2}\sum_{(u,i)\in \kappa} (r_{ui} - q_{i}^{\top} p_u)^2 + \lambda(||p_u||^2 + ||q_i||^2)
\end{equation}

where $\kappa$ is the set of $(u,i)$ pairs for which $r_{u,i}$ is known (the training set), and $\lambda$ is the regularizer parameter.

## Learning Algorithms

As described in <a href='https://ieeexplore.ieee.org/document/5197422'>(Yehuda Koren et al., 2009)</a>, to minimize the cost function $J$, the matrix factorization algorithm predicts $\hat{r}_{u,i}$ for each given training case (existing $r_{u,i}$), and computes the associated error defined by the Mean Absolute Error (MAE) as :

\begin{equation}
e_{u,i} = |r_{ui} - q_{i}^{\top} p_u|.
\end{equation}

<b>Note</b> : The overall error $E$ is defined as :

\begin{equation}
E = \frac{1}{M}\sum_{(u,i)\in\kappa} e_{u,i}
\end{equation}

Where $M$ is the number of example. The update rules for parameters $p_u$ and $q_i$ are defined as follows :

\begin{equation}
q_i \leftarrow q_i - \alpha\frac{\partial}{\partial q_i}J_{u,i}, 
\end{equation}

\begin{equation}
p_u \leftarrow p_u - \alpha\frac{\partial}{\partial p_u}J_{u,i}
\end{equation}

where $\alpha$ is the learning rate and $\frac{\partial}{\partial p_u}J_{u,i}$ is the partial derivative of the cost function $J$ according to $p_u$. It computes the extent to which $p_u$ contributes to the total error.

### How to compute $\frac{\partial}{\partial q_i}J_{u,i}$ ?

\begin{align}
\frac{\partial}{\partial q_i}J_{u,i} & = & \frac{1}{2}\frac{\partial}{\partial q_i} \begin{bmatrix}(r_{ui} - q_{i}^{\top} p_u)^2 + \lambda(||p_u||^2 + ||q_i||^2)\end{bmatrix} \\
& = & -(r_{u,i}-q_{i}^{\top} p_u)\cdot p_u + \lambda \cdot q_i  \\
& = & -e_{u,i}\cdot p_u+\lambda \cdot q_i
\end{align}

The update rules are then given by : 

\begin{equation}
q_i \leftarrow q_i + \alpha\cdot (e_{u,i}\cdot p_u-\lambda \cdot q_i), 
\end{equation}

\begin{equation}
p_u \leftarrow p_u + \alpha\cdot (e_{u,i}\cdot q_i-\lambda \cdot p_u)
\end{equation}

## Matrix Factorization : algorithm
<ol>
    <li>Initialize $P$ and $Q$ with random values
    <li>For each training example $(u,i)\in\kappa$ with the corresponding rating $r_{u,i}$ :
        <ul>
            <li>compute $\hat{r}_{u,i}$ as $\hat{r}_{u,i} = q_{i}^{\top} p_u$
            <li>compute the error : $e_{u,i} = |r_{ui} - \hat{r}_{u,i}|$
            <li>update $p_u$ and $q_i$:
                <ul>
                    <li>$p_u \leftarrow p_u + \alpha\cdot (e_{u,i}\cdot q_i-\lambda \cdot p_u)$
                    <li>$q_i \leftarrow q_i + \alpha\cdot (e_{u,i}\cdot p_u-\lambda \cdot q_i)$
                </ul>
        </ul>
    <li> Repeat step 2 until the optimal parameters are reached.
</ol>


### Download useful files

In [ ]:
import os
if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

### Import requirements
```
matplotlib==3.2.2
numpy==1.18.1
pandas==1.0.5
python==3.6.10
scikit-learn==0.23.1
scipy==1.5.0
```

In [1]:
from recsys.preprocessing import mean_ratings
from recsys.preprocessing import normalized_ratings
from recsys.preprocessing import encode_data
from recsys.preprocessing import split_data
from recsys.preprocessing import rating_matrix
from recsys.preprocessing import get_examples
from recsys.preprocessing import scale_ratings

from recsys.datasets import mlLastedSmall
from recsys.datasets import ml100k
from recsys.datasets import ml1m

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

## Model definition

In [2]:
class MatrixFactorization:
    
    def __init__(self, m, n, k=10, alpha=0.001, lamb=0.01):
        """
        Initialization of the model        
        : param
            - m : number of users
            - n : number of items
            - k : length of latent factor, both for users and items. 50 by default
            - alpha : learning rate. 0.001 by default
            - lamb : regularizer parameter. 0.02 by default
        """
        np.random.seed(32)
        
        # initialize the latent factor matrices P and Q (of shapes (m,k) and (n,k) respectively) that will be learnt
        self.k = k
        self.P = np.random.normal(size=(m, k))
        self.Q = np.random.normal(size=(n, k))
        
        # hyperparameter initialization
        self.alpha = alpha
        self.lamb = lamb
        
        # training history
        self.history = {
            "epochs":[],
            "loss":[],
            "val_loss":[],
            "lr":[]
        }
    
    def print_training_parameters(self):
        print('Training Matrix Factorization Model ...')
        print(f'k={self.k} \t alpha={self.alpha} \t lambda={self.lamb}')
    
    def update_rule(self, u, i, error):
        self.P[u] = self.P[u] + self.alpha * (error * self.Q[i] - self.lamb * self.P[u])
        self.Q[i] = self.Q[i] + self.alpha * (error * self.P[u] - self.lamb * self.Q[i])
        
    def mae(self,  x_train, y_train):
        """
        returns the Mean Absolute Error
        """
        # number of training exemples
        M = x_train.shape[0]
        error = 0
        for pair, r in zip(x_train, y_train):
            u, i = pair
            error += abs(r - np.dot(self.P[u], self.Q[i]))
        return error/M
    
    def print_training_progress(self, epoch, epochs, error, val_error, steps=5):
        if epoch == 1 or epoch % steps == 0 :
                print("epoch {}/{} - loss : {} - val_loss : {}".format(epoch, epochs, round(error,3), round(val_error,3)))
                
    def learning_rate_schedule(self, epoch, target_epochs = 20):
        if (epoch >= target_epochs) and (epoch % target_epochs == 0):
                factor = epoch // target_epochs
                self.alpha = self.alpha * (1 / (factor * 20))
                print("\nLearning Rate : {}\n".format(self.alpha))
    
    def fit(self, x_train, y_train, validation_data, epochs=1000):
        """
        Train latent factors P and Q according to the training set
        
        :param
            - x_train : training pairs (u,i) for which rating r_ui is known
            - y_train : set of ratings r_ui for all training pairs (u,i)
            - validation_data : tuple (x_test, y_test)
            - epochs : number of time to loop over the entire training set. 
            1000 epochs by default
            
        Note that u and i are encoded values of userid and itemid
        """
        self.print_training_parameters()
        
        # validation data
        x_test, y_test = validation_data
        
        # loop over the number of epochs
        for epoch in range(1, epochs+1):
            
            # for each pair (u,i) and the corresponding rating r
            for pair, r in zip(x_train, y_train):
                
                # get encoded values of userid and itemid from pair
                u,i = pair
                
                # compute the predicted rating r_hat
                r_hat = np.dot(self.P[u], self.Q[i])
                
                # compute the prediction error
                e = abs(r - r_hat)
                
                # update rules
                self.update_rule(u, i, e)
                
            # training and validation error  after this epochs
            error = self.mae(x_train, y_train)
            val_error = self.mae(x_test, y_test)
            
            # update history
            self.history['epochs'].append(epoch)
            self.history['loss'].append(error)
            self.history['val_loss'].append(val_error)
            
            # update history
            self.update_history(epoch, error, val_error)
            
            # print training progress after each steps epochs
            self.print_training_progress(epoch, epochs, error, val_error, steps=1)
              
            # leaning rate scheduler : redure the learning rate as we go deeper in the number of epochs
            # self.learning_rate_schedule(epoch)
        
        return self.history
    
    def update_history(self, epoch, error, val_error):
        self.history['epochs'].append(epoch)
        self.history['loss'].append(error)
        self.history['val_loss'].append(val_error)
        self.history['lr'].append(self.alpha)
    
    def evaluate(self, x_test, y_test):
        """
        compute the global error on the test set
        
        :param
            - x_test : test pairs (u,i) for which rating r_ui is known
            - y_test : set of ratings r_ui for all test pairs (u,i)
        """
        error = self.mae(x_test, y_test)
        print(f"validation error : {round(error,3)}")
        
        return error
      
    def predict(self, userid, itemid):
        """
        Make rating prediction for a user on an item

        :param
        - userid
        - itemid

        :return
        - r : predicted rating
        """
        # encode user and item ids to be able to access their latent factors in
        # matrices P and Q
        u = uencoder.transform([userid])[0]
        i = iencoder.transform([itemid])[0]

        # rating prediction using encoded ids. Dot product between P_u and Q_i
        r = np.dot(self.P[u], self.Q[i])

        return r

    def recommend(self, userid, N=30):
        """
        make to N recommendations for a given user

        :return 
        - (top_items,preds) : top N items with the highest predictions 
        with their corresponding predictions
        """
        # encode the userid
        u = uencoder.transform([userid])[0]

        # predictions for users userid on all product
        predictions = np.dot(self.P[u], self.Q.T)

        # get the indices of the top N predictions
        top_idx = np.flip(np.argsort(predictions))[:N]

        # decode indices to get their corresponding itemids
        top_items = iencoder.inverse_transform(top_idx)

        # take corresponding predictions for top N indices
        preds = predictions[top_idx]

        return top_items, preds        

In [3]:
epochs = 30

# 1. MovieLens Lasted Small

## Evaluation on raw ratings

In [4]:
# load data
ratings, movies = mlLastedSmall.load()

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [5]:
m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

print('number of users : ', m)
print('number of items : ', n)

number of users :  610
number of items :  9724


In [6]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 3.387 - val_loss : 3.406
epoch 2/30 - loss : 2.863 - val_loss : 2.881
epoch 3/30 - loss : 2.317 - val_loss : 2.342
epoch 4/30 - loss : 2.06 - val_loss : 2.09
epoch 5/30 - loss : 1.921 - val_loss : 1.954
epoch 6/30 - loss : 1.835 - val_loss : 1.871
epoch 7/30 - loss : 1.776 - val_loss : 1.814
epoch 8/30 - loss : 1.733 - val_loss : 1.773
epoch 9/30 - loss : 1.7 - val_loss : 1.742
epoch 10/30 - loss : 1.674 - val_loss : 1.717
epoch 11/30 - loss : 1.653 - val_loss : 1.697
epoch 12/30 - loss : 1.636 - val_loss : 1.68
epoch 13/30 - loss : 1.621 - val_loss : 1.666
epoch 14/30 - loss : 1.608 - val_loss : 1.655
epoch 15/30 - loss : 1.597 - val_loss : 1.645
epoch 16/30 - loss : 1.587 - val_loss : 1.636
epoch 17/30 - loss : 1.578 - val_loss : 1.628
epoch 18/30 - loss : 1.571 - val_loss : 1.622
epoch 19/30 - loss : 1.564 - val_loss : 1.616
epoch 20/30 - loss : 1.558 - val_loss : 1.61
epoch 21/30 - loss : 1.5

Let's visualize the learning curve

In [7]:
MF.evaluate(X_test, y_test)

validation error : 1.578


1.5779206393128034

## Evaluation on normalized ratings

In [8]:
# load data
ratings, movies = mlLastedSmall.load()

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [9]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 0.775 - val_loss : 0.783
epoch 2/30 - loss : 0.74 - val_loss : 0.746
epoch 3/30 - loss : 0.733 - val_loss : 0.74
epoch 4/30 - loss : 0.731 - val_loss : 0.739
epoch 5/30 - loss : 0.73 - val_loss : 0.738
epoch 6/30 - loss : 0.729 - val_loss : 0.738
epoch 7/30 - loss : 0.729 - val_loss : 0.738
epoch 8/30 - loss : 0.729 - val_loss : 0.738
epoch 9/30 - loss : 0.728 - val_loss : 0.738
epoch 10/30 - loss : 0.728 - val_loss : 0.738
epoch 11/30 - loss : 0.728 - val_loss : 0.738
epoch 12/30 - loss : 0.728 - val_loss : 0.738
epoch 13/30 - loss : 0.728 - val_loss : 0.738
epoch 14/30 - loss : 0.728 - val_loss : 0.738
epoch 15/30 - loss : 0.728 - val_loss : 0.738
epoch 16/30 - loss : 0.728 - val_loss : 0.738
epoch 17/30 - loss : 0.728 - val_loss : 0.738
epoch 18/30 - loss : 0.728 - val_loss : 0.738
epoch 19/30 - loss : 0.728 - val_loss : 0.738
epoch 20/30 - loss : 0.728 - val_loss : 0.738
epoch 21/30 - loss : 

In [10]:
MF.evaluate(X_test, y_test)

validation error : 0.739


0.7385341348302489

# 2. MovieLens 100k

## Evaluation on raw ratings

In [11]:
# load the ml100k dataset
ratings, movies = ml100k.load()

m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [12]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 2.712 - val_loss : 2.733
epoch 2/30 - loss : 1.767 - val_loss : 1.779
epoch 3/30 - loss : 1.597 - val_loss : 1.605
epoch 4/30 - loss : 1.543 - val_loss : 1.549
epoch 5/30 - loss : 1.52 - val_loss : 1.525
epoch 6/30 - loss : 1.508 - val_loss : 1.512
epoch 7/30 - loss : 1.501 - val_loss : 1.505
epoch 8/30 - loss : 1.497 - val_loss : 1.5
epoch 9/30 - loss : 1.493 - val_loss : 1.496
epoch 10/30 - loss : 1.491 - val_loss : 1.494
epoch 11/30 - loss : 1.489 - val_loss : 1.492
epoch 12/30 - loss : 1.488 - val_loss : 1.49
epoch 13/30 - loss : 1.487 - val_loss : 1.489
epoch 14/30 - loss : 1.486 - val_loss : 1.488
epoch 15/30 - loss : 1.485 - val_loss : 1.488
epoch 16/30 - loss : 1.485 - val_loss : 1.487
epoch 17/30 - loss : 1.484 - val_loss : 1.486
epoch 18/30 - loss : 1.484 - val_loss : 1.486
epoch 19/30 - loss : 1.483 - val_loss : 1.485
epoch 20/30 - loss : 1.483 - val_loss : 1.485
epoch 21/30 - loss : 1

In [13]:
MF.evaluate(X_test, y_test)

validation error : 1.483


1.4825599013354547

## Evaluation on normalized ratings

In [14]:
# load data
ratings, movies = ml100k.load()

m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [15]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 0.851 - val_loss : 0.85
epoch 2/30 - loss : 0.831 - val_loss : 0.831
epoch 3/30 - loss : 0.828 - val_loss : 0.828
epoch 4/30 - loss : 0.827 - val_loss : 0.828
epoch 5/30 - loss : 0.827 - val_loss : 0.827
epoch 6/30 - loss : 0.827 - val_loss : 0.827
epoch 7/30 - loss : 0.826 - val_loss : 0.827
epoch 8/30 - loss : 0.826 - val_loss : 0.827
epoch 9/30 - loss : 0.826 - val_loss : 0.827
epoch 10/30 - loss : 0.826 - val_loss : 0.827
epoch 11/30 - loss : 0.826 - val_loss : 0.827
epoch 12/30 - loss : 0.826 - val_loss : 0.827
epoch 13/30 - loss : 0.826 - val_loss : 0.827
epoch 14/30 - loss : 0.826 - val_loss : 0.827
epoch 15/30 - loss : 0.826 - val_loss : 0.827
epoch 16/30 - loss : 0.826 - val_loss : 0.827
epoch 17/30 - loss : 0.826 - val_loss : 0.827
epoch 18/30 - loss : 0.826 - val_loss : 0.827
epoch 19/30 - loss : 0.826 - val_loss : 0.827
epoch 20/30 - loss : 0.826 - val_loss : 0.827
epoch 21/30 - loss 

In [16]:
MF.evaluate(X_test, y_test)

validation error : 0.827


0.8274161711431427

# 3. MovieLens 1M

## Evaluation on raw data

In [17]:
# load the ml1m dataset
ratings, movies = ml1m.load()

m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [18]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 1.71 - val_loss : 1.719
epoch 2/30 - loss : 1.52 - val_loss : 1.526
epoch 3/30 - loss : 1.493 - val_loss : 1.498
epoch 4/30 - loss : 1.485 - val_loss : 1.489
epoch 5/30 - loss : 1.482 - val_loss : 1.486
epoch 6/30 - loss : 1.481 - val_loss : 1.484
epoch 7/30 - loss : 1.48 - val_loss : 1.483
epoch 8/30 - loss : 1.48 - val_loss : 1.483
epoch 9/30 - loss : 1.479 - val_loss : 1.483
epoch 10/30 - loss : 1.479 - val_loss : 1.482
epoch 11/30 - loss : 1.479 - val_loss : 1.482
epoch 12/30 - loss : 1.479 - val_loss : 1.482
epoch 13/30 - loss : 1.479 - val_loss : 1.482
epoch 14/30 - loss : 1.479 - val_loss : 1.482
epoch 15/30 - loss : 1.479 - val_loss : 1.482
epoch 16/30 - loss : 1.479 - val_loss : 1.482
epoch 17/30 - loss : 1.479 - val_loss : 1.482
epoch 18/30 - loss : 1.479 - val_loss : 1.482
epoch 19/30 - loss : 1.479 - val_loss : 1.482
epoch 20/30 - loss : 1.479 - val_loss : 1.482
epoch 21/30 - loss : 1

In [19]:
MF.evaluate(X_test, y_test)

validation error : 1.482


1.4815495702245565

## Evaluation on normalized ratings

In [20]:
# load data
ratings, movies = ml1m.load()

m = ratings['userid'].nunique()   # total number of users
n = ratings['itemid'].nunique()   # total number of items

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [21]:
# create the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit the model on the training set
history = MF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/30 - loss : 0.826 - val_loss : 0.826
epoch 2/30 - loss : 0.824 - val_loss : 0.825
epoch 3/30 - loss : 0.823 - val_loss : 0.825
epoch 4/30 - loss : 0.823 - val_loss : 0.825
epoch 5/30 - loss : 0.823 - val_loss : 0.825
epoch 6/30 - loss : 0.823 - val_loss : 0.825
epoch 7/30 - loss : 0.823 - val_loss : 0.825
epoch 8/30 - loss : 0.823 - val_loss : 0.825
epoch 9/30 - loss : 0.823 - val_loss : 0.825
epoch 10/30 - loss : 0.823 - val_loss : 0.825
epoch 11/30 - loss : 0.823 - val_loss : 0.825
epoch 12/30 - loss : 0.823 - val_loss : 0.825
epoch 13/30 - loss : 0.823 - val_loss : 0.825
epoch 14/30 - loss : 0.823 - val_loss : 0.825
epoch 15/30 - loss : 0.823 - val_loss : 0.825
epoch 16/30 - loss : 0.823 - val_loss : 0.825
epoch 17/30 - loss : 0.823 - val_loss : 0.825
epoch 18/30 - loss : 0.823 - val_loss : 0.825
epoch 19/30 - loss : 0.823 - val_loss : 0.825
epoch 20/30 - loss : 0.823 - val_loss : 0.825
epoch 21/30 - loss

In [22]:
MF.evaluate(X_test, y_test)

validation error : 0.825


0.8246346856565292

## Predictions

Now that the latent factors $P$ and $Q$, we can use them to make predictions and recommendations. Let's call the ```predict``` function of the ```Matrix Factorization``` class to make prediction for a given.

rating prediction for user 1 on item 6 for which the truth rating $r=4.0$

In [23]:
MF.predict(userid=1, itemid=6)

3.9034832988923427e-10

Now we can make recommendations for a  given user with function ```recommend``` of the ```Matrix Factorization``` class.

In [24]:
# get list of top N items with their corresponding predicted ratings
recommended_items, predictions = MF.recommend(userid=42)

# find corresponding movie titles
top_N = list(zip(recommended_items,predictions))
top_N = pd.DataFrame(top_N, columns=['itemid','predictions'])
List = pd.merge(top_N, movies, on='itemid', how='inner')

# show the list
List

,itemid,predictions,title,genres
0,2213,0.000016,Waltzes from Vienna (1933),Comedy|Musical
1,717,0.000016,Mouth to Mouth (Boca a boca) (1995),Comedy
2,3321,0.000015,"Waiting Game, The (2000)",Comedy
3,1118,0.000015,Tashunga (1995),Adventure|Western
4,814,0.000014,"Boy Called Hate, A (1995)",Drama
5,130,0.000014,Angela (1995),Drama
6,3291,0.000013,Trois (2000),Thriller
7,127,0.000013,"Silence of the Palace, The (Saimt el Qusur) (1...",Drama
8,572,0.000013,Foreign Student (1994),Drama
9,3904,0.000012,"Uninvited Guest, An (2000)",Drama


**Note**: The recommendation list may content items already purchased by the user. This is just an illustration of how to implement matrix factorization recommender system. You can optimize the recommended list and return the top rated items that the user has not already purchased.

## Reference

1. Yehuda Koren et al. (2009). <a href='https://ieeexplore.ieee.org/document/5197422'>Matrix Factorization Techniques for Recommender Systems</a>

## Author

<a href="https://www.linkedin.com/in/carmel-wenga-871876178/">Carmel WENGA</a>, Applied Machine Learning Research Engineer | <a href="https://shoppinglist.cm/fr/">ShoppingList</a>, Nzhinusoft